# Quickstart

The latest release of TAPE is installable via `pip`, using the following command:

```
pip install lf-tape
```

TAPE provides a scalable framework for analyzing astronomical time series data. Let's walk through a brief example where we calculate the Structure Function for a set of spectroscopically confirmed QSOs. First, we grab the available TAPE Stripe 82 QSO dataset:

In [24]:
from tape import Ensemble
ens = Ensemble() # Initialize a TAPE Ensemble
ens.from_dataset("s82_qso")

/Users/dbranton/miniforge3/envs/ray310/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54315 instead
  warnings.warn(


This dataset contains 9,258 QSOs, we can view the first 5 entries in the "object" table to get a sense of the available object-level information:

In [25]:
ens.head("object", 5)

,nobs_g,nobs_i,nobs_r,nobs_u,nobs_z,nobs_total,ra,dec,SDR5ID,M_i,M_i_corr,redshift,mass_BH,Lbol,u,g,r,i,z,Au
dbID,,,,,,,,,,,,,,,,,,,,
70,62,62,62,62,62,310,2.169302,1.238649,301,-23.901,-24.181,1.0730,0.000,0.000,20.793,20.469,20.197,20.040,20.000,0.116
98,52,52,52,52,52,260,1.091028,0.962126,144,-23.399,-23.576,0.7867,0.000,0.000,20.790,20.183,19.849,19.818,19.430,0.183
233,59,59,59,59,59,295,0.331289,0.177230,58,-24.735,-25.058,1.6199,0.000,0.000,20.892,20.554,20.431,20.199,20.099,0.154
1018,63,63,63,63,63,315,1.364696,-0.098956,190,-23.121,-24.045,0.6125,0.000,45.433,20.098,19.722,19.784,19.485,19.541,0.178
1310,59,59,59,59,59,295,0.221552,-0.292485,36,-26.451,-26.974,2.7563,9.361,46.760,20.707,19.663,19.610,19.705,19.529,0.174


The Ensemble stores data in two `dask` dataframes, object-level information in the "object" table as shown above, and individual time series measurements in the "source" table. As a result, many operations on the Ensemble closely follow operations on `dask` (and by extension `pandas`) dataframes. Let's filter down our large QSO set to a smaller set with the total number of observations per object within a certain range:

In [26]:
ens = ens.query("nobs_total >= 95 & nobs_total <= 105", "object")

We can now view the entirety of our remaining QSO set:

In [27]:
ens.compute("object")

,nobs_g,nobs_i,nobs_r,nobs_u,nobs_z,nobs_total,ra,dec,SDR5ID,M_i,M_i_corr,redshift,mass_BH,Lbol,u,g,r,i,z,Au
dbID,,,,,,,,,,,,,,,,,,,,
102187,19,19,19,19,19,95,2.815377,1.249789,406,-22.891,-23.368,0.5804,8.023,45.571,20.777,20.368,19.950,19.570,19.273,0.142
138158,21,21,21,21,21,105,10.133773,-0.230790,1541,-22.266,-22.827,0.2419,8.332,45.131,19.090,18.857,18.485,18.085,18.033,0.117
187596,19,19,19,19,19,95,10.556096,0.988253,1615,-22.652,-23.392,0.3289,8.392,45.321,19.915,19.185,18.589,18.423,17.770,0.105
711662,20,20,20,20,20,100,15.176414,1.083781,2359,-22.703,-23.310,0.6383,0.000,0.000,20.942,20.491,20.330,19.979,19.815,0.117
762267,19,19,19,19,19,95,343.360626,0.507056,75223,-22.187,-23.001,0.4627,7.856,45.186,21.107,20.647,20.193,19.855,19.529,0.465
1128581,19,19,19,19,19,95,339.200653,1.190031,74754,-24.092,-24.483,0.5481,0.000,45.471,20.410,19.395,18.889,18.333,18.346,0.398
1250783,21,21,21,21,21,105,28.063360,0.648427,4283,-23.270,-24.417,0.8656,8.432,45.550,20.865,20.368,20.131,20.169,19.936,0.158
1254675,21,21,21,21,21,105,29.243357,0.271094,4485,-22.436,-23.097,0.3593,7.834,45.277,19.277,19.202,19.007,18.873,18.387,0.161
1266724,21,21,21,21,21,105,26.613321,0.350273,4077,-22.295,-22.739,0.4051,8.177,45.200,19.993,19.623,19.428,19.300,18.959,0.154


Finally, we can calculate the Structure Function for each of these QSOs, using the available TAPE Structure Function Module:

In [28]:
from tape.analysis import calc_sf2
ens.batch(calc_sf2, sf_method="macleod_2012") # The batch function applies the provided function to all individual lightcurves within the Ensemble

,lc_id,band,dt,sf2,1_sigma
0,102187,g,212.944457,88.916180,0.0
1,102187,g,1113.928944,0.240870,0.0
2,102187,i,212.944457,0.130240,0.0
3,102187,i,1113.928944,0.132645,0.0
4,102187,r,212.944457,0.090280,0.0
...,...,...,...,...,...
5,4974467,r,1857.713042,0.202760,0.0
6,4974467,u,330.626307,1.048580,0.0
7,4974467,u,1857.713042,56.866040,0.0
8,4974467,z,330.626307,1.309430,0.0


The result is a table of delta times (dts) and structure function (sf2) for each unique lightcurve (labeled by lc_id).